# Credit Risk Assessment: Data Wrangling Pipeline

---

### Import Libraries and set Configurations

In [3]:
import os
import warnings
from pathlib import Path
import joblib
import numpy as np
import pandas as pd
import json
from datetime import date

pd.set_option("display.max_columns", 200)
warnings.filterwarnings("ignore")

random_state = 42
missing_threshold = .05
data_filepath = Path("../data/raw/accepted_2007_to_2018Q4.csv") 
models_path = Path("../models")
processed = Path("../data/processed")

### Import Dataset

In [4]:
df = pd.read_csv(data_filepath, low_memory=False).sort_values("issue_d").reset_index(drop=True)
print(f"DatasetLoaded: {data_filepath.name}")
print("Raw Shape:", df.shape)
display(df.head())

DatasetLoaded: accepted_2007_to_2018Q4.csv
Raw Shape: (2260701, 151)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,325232,NaN,9000.0,9000.0,3884.559321,36 months,9.76,289.40,B,B2,El Monte City School District,9 years,MORTGAGE,65332.0,Not Verified,Apr-2008,Does not meet the credit policy. Status:Fully ...,n,https://lendingclub.com/browse/loanDetail.acti...,I made a HUGE payment on my loan for this cred...,vacation,Worldmark the (Steal Your Money) Club,917xx,CA,17.12,0.0,Oct-1995,725.0,729.0,4.0,67.0,NaN,16.0,0.0,44248.0,22.8,41.0,f,0.0,0.0,10435.051642,4361.85,8999.99,1420.06,15.0,0.0,0.0,Apr-2011,291.58,May-2011,Apr-2011,649.0,645.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,313864,NaN,12000.0,12000.0,50.340000,36 months,12.61,402.08,D,D1,Save-A-Lot,4 years,MORTGAGE,50268.0,Not Verified,Apr-2008,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Help me get my life back! I have a full-tim...,debt_consolidation,Consolidate High Interest Credit Card,490xx,MI,10.12,0.0,Dec-1983,670.0,674.0,0.0,NaN,105.0,12.0,1.0,16083.0,70.2,31.0,f,0.0,0.0,14462.254805,53.60,12000.00,2462.26,0.0,0.0,0.0,Feb-2011,1194.22,NaN,Aug-2018,664.0,660.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [5]:
df = df[df["id"].notna()].copy() #drops rows like the footer with no id data

bad_status = {"Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off",
              "Late (31-120 days)", "Late (16-30 days)", "In Grace Period"}
good_status = {"Fully Paid","Does not meet the credit policy. Status:Fully Paid"}
df = df[df["loan_status"].isin(bad_status | good_status)].copy()
df["default"] = df["loan_status"].isin(bad_status).astype(int)
print("After mapping loan_status to default:\n", df.shape)
print("Class distribution:\n", df["default"].value_counts(normalize=True))
class_ratio = df["default"].value_counts(normalize=True).min() / df["default"].value_counts(normalize=True).max()

After mapping loan_status to default:
 (1382351, 152)
Class distribution:
 default
0    0.780365
1    0.219635
Name: proportion, dtype: float64


### Datetime Conversion

In [6]:
date_cols = ["issue_d","earliest_cr_line","last_pymnt_d","next_pymnt_d",
                 "last_credit_pull_d","final_pymnt_d"]
for col in set(date_cols) & set(df.columns):
    df[col] = pd.to_datetime(df[col], format="%b-%Y", errors="coerce")

### EDA

In [7]:
print("Column count:", len(df.columns))
display(df.info(memory_usage="deep"))

Column count: 152
<class 'pandas.core.frame.DataFrame'>
Index: 1382351 entries, 0 to 2260650
Columns: 152 entries, id to default
dtypes: datetime64[ns](5), float64(113), int64(1), object(33)
memory usage: 3.3 GB


None

In [8]:
#Check for null values
df.isnull().sum()

id                             0
member_id                1382351
loan_amnt                      0
funded_amnt                    0
funded_amnt_inv                0
                          ...   
settlement_date          1348133
settlement_amount        1348133
settlement_percentage    1348133
settlement_term          1348133
default                        0
Length: 152, dtype: int64

In [9]:
#Check for duplicates
df.duplicated().sum()

0

In [10]:
#Descriptive statistics
df.describe().T

,count,mean,min,25%,50%,75%,max,std
member_id,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan_amnt,1382351.0,14477.704776,500.0,8000.0,12000.0,20000.0,40000.0,8753.375197
funded_amnt,1382351.0,14469.11121,500.0,8000.0,12000.0,20000.0,40000.0,8749.667152
funded_amnt_inv,1382351.0,14442.02215,0.0,7950.0,12000.0,20000.0,40000.0,8755.722826
int_rate,1382351.0,13.299137,5.31,9.75,12.79,16.02,30.99,4.79338
...,...,...,...,...,...,...,...,...
hardship_last_payment_amount,8004.0,193.600027,0.01,46.25,132.08,281.265,1407.86,197.940829
settlement_amount,34218.0,5010.126248,44.21,2208.0,4147.0,6850.1725,33601.0,3690.532108
settlement_percentage,34218.0,47.77095,0.2,45.0,45.0,50.0,521.35,7.304528
settlement_term,34218.0,13.191712,0.0,6.0,14.0,18.0,181.0,8.162197


In [11]:
print("Numerical columns:")
df.describe(include=[np.number])

Numerical columns:


,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term,default
count,0.0,1.382351e+06,1.382351e+06,1.382351e+06,1.382351e+06,1.382351e+06,1.382347e+06,1.381939e+06,1.382322e+06,1.382351e+06,1.382351e+06,1.382321e+06,686489.000000,235619.000000,1.382322e+06,1.382322e+06,1.382351e+06,1.381416e+06,1.382322e+06,1.382351e+06,1.382351e+06,1.382351e+06,1.382351e+06,1.382351e+06,1.382351e+06,1.382351e+06,1.382351e+06,1.382351e+06,1.382351e+06,1.382351e+06,1.382351e+06,1.382206e+06,364175.000000,1382351.0,2.958400e+04,29581.000000,1.382322e+06,1.312075e+06,1.312075e+06,569195.000000,569196.000000,569196.000000,569196.000000,553845.000000,5.691960e+05,491955.000000,569196.000000,569196.000000,569196.000000,569137.000000,1.312075e+06,569196.000000,569195.000000,569195.000000,1.332321e+06,1.312052e+06,1.317980e+06,1.317190e+06,1.382206e+06,1.382322e+06,1.272839e+06,1.312074e+06,1.312074e+06,1.312075e+06,1.332321e+06,1.318939e+06,327833.000000,1.202588e+06,462405.000000,1.312075e+06,1.312075e+06,1.312075e+06,1.323761e+06,1.312075e+06,1.312075e+06,1.312075e+06,1.312074e+06,1.312075e+06,1.323761e+06,1.260121e+06,1.312075e+06,1.312075e+06,1.312075e+06,1.311921e+06,1.317566e+06,1.380986e+06,1.382246e+06,1.312075e+06,1.332321e+06,1.332321e+06,1.312075e+06,22091.000000,22092.000000,22092.000000,22092.000000,22092.000000,22092.000000,21697.000000,22092.000000,22092.000000,22092.000000,22092.000000,8103.000000,8004.0,8004.000000,8004.0,8004.000000,5819.000000,8004.000000,8004.000000,34218.000000,34218.000000,34218.000000,1.382351e+06
mean,NaN,1.447770e+04,1.446911e+04,1.444202e+04,1.329914e+01,4.394854e+02,7.629012e+04,1.832105e+01,3.190769e-01,6.960639e+02,7.000640e+02,6.608718e-01,34.268517,70.512866,1.159196e+01,2.154115e-01,1.625139e+04,5.178038e+01,2.493117e+01,2.782387e+02,2.781766e+02,1.467812e+04,1.465013e+04,1.201745e+04,2.423437e+03,1.935405e+00,2.352969e+02,3.922059e+01,5.295305e+03,6.758941e+02,6.587925e+02,1.728179e-02,43.698282,1.0,1.176054e+05,19.072656,5.013304e-03,2.489284e+02,1.409653e+05,1.050115,2.791097,0.774206,1.739475,19.632612,3.590755e+04,71.400154,1.421993,3.008776,5522.336469,58.244166,3.275134e+04,1.099403,1.603240,2.316401,4.700781e+

In [12]:
#Check for object columns
print("Object columns:")
df.describe(include=["object"]).T

Object columns:


,count,unique,top,freq
id,1382351,1382351,325232,1
term,1382351,2,36 months,1043059
grade,1382351,7,B,400654
sub_grade,1382351,35,C1,87840
emp_title,1293206,385454,Teacher,21799
emp_length,1300914,11,10+ years,453281
home_ownership,1382351,6,MORTGAGE,682142
verification_status,1382351,3,Source Verified,536031
loan_status,1382351,8,Fully Paid,1076751
pymnt_plan,1382351,2,n,1381734


### Temporal Split

In [13]:
cutoff_train = pd.Timestamp("2016-12-31")
cutoff_val = pd.Timestamp("2017-12-31")
train_mask = df["issue_d"] <= cutoff_train
val_mask = (df["issue_d"] > cutoff_train) & (df["issue_d"] <= cutoff_val)
test_mask = df["issue_d"] > cutoff_val
print("Train shape:", df[train_mask].shape)
print("Validation shape:", df[val_mask].shape)
print("Test shape:", df[test_mask].shape)

Train shape: (1132562, 152)
Validation shape: (181728, 152)
Test shape: (68061, 152)


### Data Cleaning

In [14]:
#Drop columns that are irrelevant to the model or result in high leakage
drop_exact = [
    "loan_status","pymnt_plan","member_id","id","url",
    "addr_state","zip_code","emp_title","title","desc"
]
drop_prefix = [
    "last_",                 
    "out_prncp",             
    "total_rec_",            
    "total_pymnt",           
    "recoveries",
    "collection_recovery_fee",
    "funded_amnt_inv",       
    "hardship_",             
    "debt_settlement_",      
    "settlement_"            
]
leak_cols = set(drop_exact)
for p in drop_prefix:
    leak_cols |= {c for c in df.columns if c.startswith(p)}
df.drop(columns=leak_cols & set(df.columns), inplace=True)

print(f"New shape is: {df.shape}")

New shape is: (1382351, 108)


### Convert Percent Strings

In [15]:
pct_cols = ["int_rate", "revol_util"]
for col in pct_cols:
    if col in df.columns:
        df[col]=pd.to_numeric(df[col].astype(str).str.rstrip("%"),errors="coerce")/100

### Log Transform

In [16]:
for c in ("loan_amnt","annual_inc","revol_bal"):
    if c in df.columns:
        df[c] = np.log1p(df[c])

### Missing Flag

In [17]:
for col in df.columns.difference(["default"]):
    if df.loc[train_mask, col].isna().mean() > missing_threshold:
        df[f"{col}_missing"] = df[col].isna().astype("int8")

print(f"New shape is: {df.shape}")

New shape is: (1382351, 178)


### Imputation, Encoding, Missing Flags

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.difference(['default'])
cat_cols = df.select_dtypes("object").columns.tolist()

df[num_cols] = df[num_cols].astype(np.float32)
df[num_cols] = df[num_cols].fillna(df.loc[train_mask, num_cols].median())

for c in cat_cols:
    mode_val = (df.loc[train_mask, c].mode(dropna=True).iat[0] if not df.loc[train_mask, c]
                                   .mode(dropna=True).empty else "Missing")
    df[c] = df[c].fillna(mode_val)
    df[c] = df[c].astype("category")
    extra = [x for x in ("Missing", "Other") if x not in df[c].cat.categories]
    if extra:
        df[c] = df[c].cat.add_categories(extra)
    cutoff = max(10, 0.001 * train_mask.sum())
    rare = (df.loc[train_mask, c]
            .value_counts()[lambda s: s < cutoff]
            .index.difference(["Missing","Other"]))
    df.loc[df[c].isin(rare), c] = "Other"
    df[c] = df[c].cat.remove_unused_categories()

print("shape:", df.shape)

shape: (1382351, 178)


### Feature Engineering

In [ ]:
if {"issue_d","earliest_cr_line"} <= set(df.columns):
    df["credit_age_months"] = (
        (df["issue_d"] - df["earliest_cr_line"]).dt.days // 30)

if {"loan_amnt","annual_inc"} <= set(df.columns):
    df["loan_to_income"] = df["loan_amnt"] / (df["annual_inc"] + 1)

if {"installment","annual_inc"} <= set(df.columns):
    df["installment_to_income"] = df["installment"] / (df["annual_inc"] + 1)

if {"fico_range_low", "fico_range_high"} <= set(df.columns):
    df["fico_mid"] = (df["fico_range_low"] + df["fico_range_high"]) / 2
    df["fico_spread"] = df["fico_range_high"] - df["fico_range_low"]
    df.drop(columns=["fico_range_low", "fico_range_high"], inplace=True)

if {"revol_bal", "total_rev_hi_lim"} <= set(df.columns):
    df["rev_util_ratio"] = df["revol_bal"] / (df["total_rev_hi_lim"] + 1)

if "dti" in df.columns:
    df["dti_inv"] = 1 / (df["dti"] + 1e-3)

if {"inq_last_12m", "open_acc"} <= set(df.columns):
    df["inq_ratio"] = df["inq_last_12m"] / (df["open_acc"] + 1)

if {"int_rate", "sub_grade"} <= set(df.columns):
    sg_mean = df.loc[train_mask].groupby("sub_grade")["int_rate"].transform("mean")
    df["int_minus_subgrade_mean"] = df["int_rate"] - sg_mean

if {"grade","term"}<=set(df.columns):
    df["grade_term"] = df["grade"].astype(str)+"_"+df["term"].astype(str)
    df["grade_term"]= df["grade_term"].astype("category")

if {"purpose","emp_length"}<=set(df.columns):
    df["purpose_emp_len"] = df["purpose"].astype(str)+"_"+df["emp_length"].astype(str)
    df["purpose_emp_len"] = df["purpose_emp_len"].astype("category")

if "fico_mid" in df.columns:
    df["fico_mid_sq"] = df["fico_mid"] ** 2

if "loan_amnt" in df.columns:
    df["log_loan_sq"] = (df["loan_amnt"]) ** 2

for fld in ["policy_code","initial_list_status"]:
    if fld in df and df[fld].dtype=="object":
        df[fld] = df[fld].astype("category")
        if fld not in cat_cols: cat_cols.append(fld)        

print("Features Engineered")
print("New shape is:", df.shape)

Features Engineered
New shape is: (1382351, 189)


### Drop Constant Columns

In [20]:
constant_cols = [c for c in df.columns if df[c].nunique(dropna=False) == 1]
if constant_cols:
    print("dropping constant features:", constant_cols)
    df.drop(columns=constant_cols, inplace=True)

print("New shape is:", df.shape)

dropping constant features: ['policy_code', 'deferral_term']
New shape is: (1382351, 187)


### Save Cleaned/Engineered Data

In [21]:
df.to_parquet("../data/processed/lc_cleaned.parquet")

schema = {
    "numeric": sorted(df.select_dtypes(include=[np.number]).columns.difference(["default"])),
    "categorical": sorted(df.select_dtypes("category").columns),
    "target": "default",
    "date_split_col": "issue_d"
}
json.dump(schema, open("../data/processed/feature_schema.json","w"), indent=2)

def save_split(mask, name):
    X, y = df.loc[mask].drop("default",axis=1), df.loc[mask,"default"]
    X.to_pickle(f"../data/processed/X_{name}.pkl")
    y.to_csv (f"../data/processed/y_{name}.csv", index=False)
for msk,name in [(train_mask,"train"),(val_mask,"val"),(test_mask,"test")]:
    save_split(msk,name)
print("Pre processing complete – data written to", "../data/processed/")

Pre processing complete – data written to ../data/processed/


In [25]:
df = pd.read_parquet(processed / "lc_cleaned.parquet")
schema = json.load(open(processed / "feature_schema.json"))
X_train= pd.read_pickle(processed / "X_train.pkl")
X_val = pd.read_pickle(processed / "X_val.pkl")
X_test = pd.read_pickle(processed / "X_test.pkl")
y_train= pd.read_csv(processed / "y_train.csv")["default"]

train_rows = len(X_train)
na_thr = 0.05
rare_min = max(10, int(0.001 * train_rows))
imb_ratio = round((y_train == 0).sum() / (y_train == 1).sum(), 2)

readme = f"""\
# Lending Club — Data Preprocessing Pipeline

_Generated **{date.today()}**_

---

## 1  Raw Input
**{data_filepath.name}** — original Lending Club “accepted loans” CSV (2007–2018 Q4).

---

## 2  Target Definition
| Label | Mapped `loan_status` values |
|-------|-----------------------------|
| **1** | Charged Off, Default, Late (16–30 d), Late (31–120 d), In Grace Period, Does-Not-Meet-Policy Charged Off |
| **0** | Fully Paid, Does-Not-Meet-Policy Fully Paid |

Other statuses were **dropped**.  
Default rate (train split): **{y_train.mean():.1%}**  
Class imbalance ≈ **{imb_ratio}:1** (non-default : default)

---

## 3  Column Filtering (Leakage, PII)
* **Exact drops:** `loan_status`, `pymnt_plan`, `member_id`, `id`, `url`, `addr_state`, `zip_code`, `emp_title`, `title`, `desc`
* **Prefix drops:** any column that starts with  
  `last_`, `out_prncp`, `total_rec_`, `total_pymnt`, `recoveries`,  
  `collection_recovery_fee`, `funded_amnt_inv`, `hardship_`,  
  `debt_settlement_`, `settlement_`
* **Constant columns** (`nunique()==1`) removed.
* **High-missing flag rule:** if `train` NaN rate ≥ **{na_thr:.0%}**, create `<col>_missing` indicator ( dtype int8 ).

_Post-filter dataframe shape:_ **{df.shape}**

---

## 4  Temporal Split (`issue_d`)
| Split | Date window               | Rows |
|-------|---------------------------|------|
| Train | ≤ 2016-12-31              | {len(X_train):,} |
| Val   | 2017-01-01 → 2017-12-31   | {len(X_val):,} |
| Test  | ≥ 2018-01-01              | {len(X_test):,} |

No random shuffling — ensures forward-looking evaluation.

---

## 5  Pre-processing Steps
### 5.1  Type casting & basic transforms
* **Percent to float**: `int_rate`, `revol_util` → divide by 100.  
* **Log1p**: `loan_amnt`, `annual_inc`, `revol_bal` to reduce skew.

### 5.2  Missing-value handling
* **Numeric** (`{len(schema['numeric'])}` cols) → median of *train* split.  
* **Categorical** (`{len(schema['categorical'])}` cols)  
  * NaNs → `"Missing"` level (added if absent).  
  * **Rare bucketing:** any level with < max(10, 0.1 %) occurrences in *train* → `"Other"`.

### 5.3  Feature Engineering
| Feature | Formula | Notes |
|---------|---------|-------|
| `credit_age_months` | `(issue_d – earliest_cr_line)//30` | Age of credit file |
| `loan_to_income` | `loan_amnt / (annual_inc+1)` | Ratio |
| `installment_to_income` | `installment / (annual_inc+1)` | Ratio |
| `fico_mid`, `fico_spread` | Midpoint & range of approval-time FICO | |
| `rev_util_ratio` | `revol_bal / (total_rev_hi_lim+1)` | |
| `dti_inv` | `1 / (dti+1e-3)` | Stabilised inverse |
| `inq_ratio` | `inq_last_12m / (open_acc+1)` | Hard-pull density |
| `int_minus_subgrade_mean` | `int_rate – mean(int_rate) within sub_grade` (calc **on train only**) | Removes grade-level bias |
| `grade_term` | concat(`grade`, `term`) | Categorical cross |
| `purpose_emp_len` | concat(`purpose`, `emp_length`) | High-card cat |
| `fico_mid_sq`, `log_loan_sq` | Squared versions for non-linearity |

Total feature count after engineering: **{df.drop(columns=['default']).shape[1]}**

---

## 6  Encoding & Data Types
* All categoricals kept as native **pandas `category`** → consumed directly by LightGBM.  
* No one-hot nor target-encoding currently applied (reserved for later versions).  

---

## 7  Saved Artefacts
| File | Purpose |
|------|---------|
| `lc_cleaned.parquet` | Full cleaned dataset (train + val + test) |
| `X_train.pkl`, `X_val.pkl`, `X_test.pkl` | Feature DataFrames |
| `y_train.csv`, `y_val.csv`, `y_test.csv` | Target vectors |
| `feature_schema.json` | Lists numeric / categorical columns |
| `README.md` | This manifest |

---

## 8  Reproducibility Guarantees
* **Leakage-proof**: all post-issuance columns removed _before_ split.  
* **Train-only statistics** for imputation & mean-subtractions.  
* **Monotonic date split** prevents future information bleed.  
* Every artefact versioned under `../data/processed/`. """ 

(processed / "README.md").write_text(readme, encoding="utf-8")
print("README.md updated →", (processed / 'README.md').resolve())

README.md updated → /Users/pranavrao/Documents/ai-ml-projects/github-repos/aura-xai-finrisk-llm/data/processed/README.md
